In [6]:
import numpy as np

covs_all = np.load('../data/simulation2/covs_all.npy')
cov = np.load('../data/simulation2/cov_pop.npy')

In [7]:
num_cells = 10
len_series = np.array([68, 116, 216, 216,  94, 216, 216, 190, 216, 216],dtype=np.int64)

In [8]:
ns12 = np.zeros((34,11),dtype=np.int64)

In [9]:
list0 = [0,3,4,6,7,9,10,11,12,13,15,18,19,20]
list1 = [2,7,8,9,12,14,15,16,17,18,20,24,25,26,28,31,32,33]
list2 = [2,4,5,6,7,8,9,10]

In [10]:
for i in list1:
    for j in list2:
        print(i,j)
        eigvals = np.zeros((num_cells,2))
        sqrt_eigvals = np.zeros((num_cells,2))
        eigvecs = np.zeros((num_cells,2,2))
        for c in range(num_cells):
            #mat = np.array([[S00[c],covs[c,0,1],covs[c,0,2],covs[c,0,6]],[covs[c,1,0],covs[c,1,1],covs[c,1,2],covs[c,1,6]],[covs[c,2,0],covs[c,2,1],covs[c,2,2],covs[c,2,6]],[covs[c,6,0],covs[c,6,1],covs[c,6,2],covs[c,6,6]]])
            #eigvals[c], eigvecs[c] = np.linalg.eigh(mat)
            eigvals[c], eigvecs[c] = np.linalg.eigh(covs_all[np.ix_([c],[i+25,j+59],[i+25,j+59])])
            sqrt_eigvals[c] = np.sqrt(eigvals[c])

        num_trials = 5000000
        vals = np.zeros((num_trials,num_cells,2))
        for tr in range(num_trials):
            #if tr%10000==0: print(tr)
            for c in range(num_cells):
                vals[tr,c] = eigvecs[c]@(sqrt_eigvals[c]*np.random.normal(size=(2,)))
        
        means_sim = np.zeros((num_trials,2))
        means_sim = np.sum(vals*len_series[np.newaxis,:,np.newaxis], axis=1)/np.sum(len_series[:])
        
        covs_sim = np.zeros((num_trials,2,2))
        for k in range(2):
            for l in range(2):
                covs_sim[:,k,l] = np.sum(len_series[np.newaxis,:]*(vals[:,:,k]-means_sim[:,np.newaxis,k])*(vals[:,:,l]-means_sim[:,np.newaxis,l]), axis=1)/np.sum(len_series[:])
        covs_diff = cov[np.ix_([i+25,j+59],[i+25,j+59])][np.newaxis,:,:] - covs_sim
        
        #scale_mat_inv = np.linalg.inv(scale_mat[np.ix_([i,j],[i,j])])
        #mat_sim = np.matmul(covs_diff, scale_mat_inv)
        a = covs_diff[:,0,0]
        b = covs_diff[:,0,1]
        c = covs_diff[:,1,0]
        d = covs_diff[:,1,1]
        eigs_sim = (a+d-np.sqrt((a-d)**2+4*b*c))/2
        ns12[i,j] = np.sum(np.where(eigs_sim<=0,1,0))
        print(i,j,ns12[i,j])

2 2
2 2 1514970
2 4
2 4 66
2 5
2 5 3026026
2 6
2 6 194
2 7
2 7 1952
2 8
2 8 0
2 9
2 9 225463
2 10
2 10 0
7 2
7 2 34925
7 4
7 4 12198
7 5
7 5 523093
7 6
7 6 54779
7 7
7 7 295801
7 8
7 8 10978
7 9
7 9 611280
7 10
7 10 20199
8 2
8 2 1
8 4
8 4 669
8 5
8 5 596660
8 6
8 6 251998
8 7
8 7 114416
8 8
8 8 20
8 9
8 9 135096
8 10
8 10 94900
9 2
9 2 2472
9 4
9 4 25522
9 5
9 5 402090
9 6
9 6 63330
9 7
9 7 235612
9 8
9 8 9370
9 9
9 9 112865
9 10
9 10 47543
12 2
12 2 45976
12 4
12 4 348
12 5
12 5 4365857
12 6
12 6 1898471
12 7
12 7 2391
12 8
12 8 7
12 9
12 9 3635398
12 10
12 10 1341735
14 2
14 2 56968
14 4
14 4 229948
14 5
14 5 2638001
14 6
14 6 899151
14 7
14 7 306235
14 8
14 8 72973
14 9
14 9 1854475
14 10
14 10 573635
15 2
15 2 0
15 4
15 4 0
15 5
15 5 5
15 6
15 6 0
15 7
15 7 798
15 8
15 8 0
15 9
15 9 0
15 10
15 10 0
16 2
16 2 0
16 4
16 4 4617
16 5
16 5 3
16 6
16 6 0
16 7
16 7 15737
16 8
16 8 460
16 9
16 9 0
16 10
16 10 0
17 2
17 2 0
17 4
17 4 36425
17 5
17 5 53
17 6
17 6 105
17 7
17 7 37724
17 8
17

In [11]:
with open('../data/simulation2/ns12.npy','wb') as f:
    np.save(f, ns12)